In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 = input image channel, 6 = output channel, 3x3 = square convolution
        # KERNEL
        self.conv1 = nn.Conv2d(1,6,3)
        self.conv2 = nn.Conv2d(6,16,3)
        
        # y = Wx+b, 6x6 image dimension
        self.fc1 = nn.Linear(16*6*6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2,2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        # All dimensions except the batch dimension
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        
        return num_features
    

net = Net()
print(net)
        
        

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [8]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1

10
torch.Size([6, 1, 3, 3])


In [9]:
input = torch.randn(1,1,32,32)
out = net(input)
print(out)

tensor([[ 0.0261, -0.0062, -0.0663,  0.0616,  0.0826, -0.0390,  0.0128, -0.0664,
         -0.0827,  0.0041]], grad_fn=<AddmmBackward>)


In [10]:
net.zero_grad()
out.backward(torch.randn(1,10))

In [28]:
# Calculating loss

output = net(input)
# A dummy target
target = torch.randn(10)
# Set as same shape as output
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.3348, grad_fn=<MseLossBackward>)


In [18]:
# loss.backward()

In [29]:
print(loss.grad_fn) # MESLoss
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLU

In [25]:
# loss.backward()

In [26]:
net.zero_grad()

In [30]:
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0254, -0.0023, -0.0050,  0.0013, -0.0104, -0.0026])


In [31]:
# Optimizer

import torch.optim as optim

# create optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop
# zero the gradient buffer
optimizer.zero_grad()
opt_output = net(input)
opt_loss = criterion(opt_output, target)
opt_loss.backward()

# does update
optimizer.step()